<a href="https://colab.research.google.com/github/nitpat25/Object-Detection/blob/main/1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

[Errno 20] Not a directory: 'darknet'
/content/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:945:23: warning: variable ‘rgb’ set but not used []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-but-set-variable-Wunused-but-set-variable]8;;]
  945 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1443:14: warning: unused variab

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "Koala" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

mkdir: cannot create directory ‘data/obj’: File exists


In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2024-02-09 13:38:15--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.1’

darknet53.conv.74.1 100%[===================>] 154.96M  48.6MB/s    in 3.8s    

2024-02-09 13:38:19 (40.4 MB/s) - ‘darknet53.conv.74.1’ saved [162482580/162482580]



In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/Screenshot 2024-01-29 202938.png  
  inflating: data/obj/Screenshot 2024-01-29 202938.txt  
  inflating: data/obj/Screenshot 2024-01-29 203000.png  
  inflating: data/obj/Screenshot 2024-01-29 203000.txt  
  inflating: data/obj/Screenshot 2024-01-29 203021.png  
  inflating: data/obj/Screenshot 2024-01-29 203021.txt  
  inflating: data/obj/Screenshot 2024-01-29 203045.png  
  inflating: data/obj/Screenshot 2024-01-29 203045.txt  
  inflating: data/obj/Screenshot 2024-01-29 203104.png  
  inflating: data/obj/Screenshot 2024-01-29 203104.txt  
  inflating: data/obj/Screenshot 2024-01-29 203135.png  
  inflating: data/obj/Screenshot 2024-01-29 203135.txt  
  inflating: data/obj/Screenshot 2024-01-29 203208.png  
  inflating: data/obj/Screenshot 2024-01-29 203208.txt  
  inflating: data/obj/Screenshot 2024-01-29 203229.png  
  inflating: data/obj/Screenshot 2024-01-29 203229.txt  
  inflating: data/obj/Screenshot 2024-01-29 203248.

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.550000 0.508621 0.874468 0.890805

['15', '0.550000', '0.508621', '0.874468', '0.890805']
0 data/obj/Screenshot 2024-01-29 203549.txt
0 0.550000 0.508621 0.874468 0.890805
15 0.537363 0.425076 0.652747 0.764526

['15', '0.537363', '0.425076', '0.652747', '0.764526']
1 data/obj/Screenshot 2024-01-29 204352.txt
0 0.537363 0.425076 0.652747 0.764526
15 0.541367 0.459794 0.787770 0.911340

['15', '0.541367', '0.459794', '0.787770', '0.911340']
2 data/obj/Screenshot 2024-01-29 203229.txt
0 0.541367 0.459794 0.787770 0.911340
15 0.543807 0.501104 0.682779 0.958057

['15', '0.543807', '0.501104', '0.682779', '0.958057']
3 data/obj/Screenshot 2024-01-29 203135.txt
0 0.543807 0.501104 0.682779 0.958057
15 0.527378 0.556061 0.893372 0.772727

['15', '0.527378', '0.556061', '0.893372', '0.772727']
4 data/obj/Screenshot 2024-01-29 204314.txt
0 0.527378 0.556061 0.893372 0.772727
15 0.525597 0.556277 0.634812 0.852814

['15', '0.525597', '0.556277', '0.634812', '0.852814']
5 data/obj/Screensho

In [ ]:
import glob
images_list = glob.glob("data/obj/*.png")
print(images_list)

['data/obj/Screenshot 2024-01-29 204600.png', 'data/obj/Screenshot 2024-01-29 204202.png', 'data/obj/Screenshot 2024-01-29 204338.png', 'data/obj/Screenshot 2024-01-29 203549.png', 'data/obj/Screenshot 2024-01-29 203208.png', 'data/obj/Screenshot 2024-01-29 204116.png', 'data/obj/Screenshot 2024-01-29 203334.png', 'data/obj/Screenshot 2024-01-29 204352.png', 'data/obj/Screenshot 2024-01-29 203915.png', 'data/obj/Screenshot 2024-01-29 203135.png', 'data/obj/Screenshot 2024-01-29 204042.png', 'data/obj/Screenshot 2024-01-29 204720.png', 'data/obj/Screenshot 2024-01-29 203402.png', 'data/obj/Screenshot 2024-01-29 204702.png', 'data/obj/Screenshot 2024-01-29 204925.png', 'data/obj/Screenshot 2024-01-29 203535.png', 'data/obj/Screenshot 2024-01-29 204314.png', 'data/obj/Screenshot 2024-01-29 203657.png', 'data/obj/Screenshot 2024-01-29 204413.png', 'data/obj/Screenshot 2024-01-29 203856.png', 'data/obj/Screenshot 2024-01-29 204450.png', 'data/obj/Screenshot 2024-01-29 204216.png', 'data/obj

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w")
file.write("\n".join(images_list))
file.close()

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.923068), count: 4, class_loss = 0.000012, iou_loss = 0.032546, total_loss = 0.032558 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 125036, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.910462), count: 4, class_loss = 0.000088, iou_loss = 0.029300, total_loss = 0.029388 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.0